# Problemas de vivacidad

## Interbloqueo
Clase con todas las funciones sincronizadas

In [ ]:
import threading
import time
class Celda:
    def __init__(self, valor,cerrojo):
        self.valor=valor
        self.cerrojo=cerrojo
        
    def obtenerValor(self):
        
        self.cerrojo.acquire()
        valor=self.valor
        print(threading.currentThread().getName()+" obtiene valor "+str(self.valor))
        self.cerrojo.release()
        
        return valor


    def ponerValor(self, valor):
        
        self.cerrojo.acquire()
        self.valor=valor
        print(threading.currentThread().getName()+" pone valor "+str(valor))
        self.cerrojo.release()
   
    def intercambiarValor(self, celda):
        self.cerrojo.acquire()
        val1=self.obtenerValor()
        val2=celda.obtenerValor()
        time.sleep(3)
        self.ponerValor(val2)
        celda.ponerValor(val1)
        self.cerrojo.release()
        
        
        
        
if __name__=="__main__":
    cerrojo1=threading.RLock()
    cerrojo2=threading.RLock()
    celda1=Celda(2,cerrojo1)
    celda2=Celda(3,cerrojo2)
    print("inicio programa")
    thread1 = threading.Thread(target=celda1.intercambiarValor, args=(celda2,))
    thread2 = threading.Thread(target=celda2.intercambiarValor, args=(celda1,))
    
    thread1.start()
    
    thread2.start()
   
    thread1.join()
    thread2.join()
    
    print("final programa"+str(celda1.obtenerValor())+" "+str(celda2.obtenerValor()))
        
        

inicio programa
Thread-4 obtiene valor 2
Thread-4 obtiene valor 3
Thread-5 obtiene valor 3
Thread-4 pone valor 3


El  problema se resuelve elminando la sincronización de la función intercambiarValor

In [1]:
import threading
import time
class Celda:
    def __init__(self, valor,cerrojo):
        self.valor=valor
        self.cerrojo=cerrojo
        
    def obtenerValor(self):
        
        self.cerrojo.acquire()
        valor=self.valor
        print(threading.currentThread().getName()+" obtiene valor "+str(self.valor))
       
        self.cerrojo.release()
        return valor


    def ponerValor(self, valor):
        
        self.cerrojo.acquire()
        self.valor=valor
        print(threading.currentThread().getName()+" pone valor "+str(valor))
        self.cerrojo.release()
   
    def intercambiarValor(self, celda):
        
        val1=self.obtenerValor()
        val2=celda.obtenerValor()
        
        self.ponerValor(val2)
        celda.ponerValor(val1)
        
        
        
        
        
if __name__=="__main__":
    cerrojo1=threading.RLock()
    cerrojo2=threading.RLock()
    celda1=Celda(2,cerrojo1)
    celda2=Celda(3,cerrojo2)
    print("inicio programa")
    thread1 = threading.Thread(target=celda1.intercambiarValor, args=(celda2,))
    thread2 = threading.Thread(target=celda2.intercambiarValor, args=(celda1,))
    thread3 = threading.Thread(target=celda1.intercambiarValor, args=(celda2,))
    thread1.start()
    
    thread2.start()
    thread3.start()
   
    thread1.join()
    thread2.join()
    thread3.join()
    
    print("final programa"+str(celda1.obtenerValor())+" "+str(celda2.obtenerValor()))
        
        

inicio programa
Thread-4 obtiene valor 2
Thread-4 obtiene valor 3
Thread-4 pone valor 3
Thread-4 pone valor 2
Thread-5 obtiene valor 2
Thread-5 obtiene valor 3
Thread-5 pone valor 3
Thread-5 pone valor 2
Thread-6 obtiene valor 2
Thread-6 obtiene valor 3
Thread-6 pone valor 3
Thread-6 pone valor 2
MainThread obtiene valor 3
MainThread obtiene valor 2
final programa3 2


## Monitores anidados

In [ ]:
import threading
class Interna:
    def __init__(self,condicion):
        self.condicion=condicion
        
    def esperarCondicion(self):
        with self.condicion:
            self.condicion.wait()
                
    def liberarCondicion(self):
        with self.condicion:
            self.condicion.notifyAll()
            
class Externa:
    def __init__(self,cerrojo,objetoInterno):
        self.objetoInterno=objetoInterno
        self.cerrojo=cerrojo
        
    def esperar(self):
        with self.cerrojo:
            self.objetoInterno.esperarCondicion()
        
    def liberar(self):
        with self.cerrojo:
            self.objetoInterno.liberarCondicion()
            
if __name__=="__main__":
    cerrojo=threading.Lock()
    condicion=threading.Condition()
    objInterno=Interna(condicion)
    objExterno=Externa(cerrojo,objInterno)
    print("inicio programa")
    thread1 = threading.Thread(target=objExterno.esperar)
    thread2 = threading.Thread(target=objExterno.liberar)
    
    thread1.start()
    
    thread2.start()
   
    thread1.join()
    thread2.join()
    
    print("final programa")
        
            

inicio programa
